In [9]:
import requests
import bs4
import pandas as pd
from selenium.webdriver import Chrome
from selenium .webdriver.common.by import By
import time
from tqdm.notebook import tqdm
import warnings
warnings.simplefilter('ignore')

In [28]:
query = "엘지"

titles = []
prices = []
review_counts = []
buy_counts = []
published_dates = []
favorites = []

driver = Chrome('./chromedriver')

for page_no in range(1, 6):

    page_url = f"https://search.shopping.naver.com/search/all?frm=NVSCTAB&origQuery={query}&pagingIndex={page_no}&pagingSize=40&productSet=total&query={query}&sort=rel&timestamp=&viewType=list"
    driver.get(page_url)
    time.sleep(1)

    for scroll_down in range(7):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.5)

    list_basis = driver.find_element(By.CLASS_NAME, "list_basis")
    item_list = list_basis.find_elements(By.CLASS_NAME, "basicList_inner__xCM3J")
    
    for i in range(len(item_list)):
        item = item_list[i]
        title = item.find_element(By.CLASS_NAME, "basicList_title__VfX3c")
        titles.append(title.text)

        price = item.find_element(By.CLASS_NAME, "price_num__S2p_v").text[:-1].replace(',', '')
        prices.append(price)

        footer = item.find_element(By.CLASS_NAME, "basicList_etc_box__5lkgg")
        reviews = footer.find_elements(By.CLASS_NAME, "basicList_num__sfz3h")
        footer_text = footer.text
        try:
            if "구매건수" in footer_text:
                review_counts.append(int(reviews[0].text.replace(',', '')))
                buy_counts.append(int(reviews[1].text.replace(',', '')))
                favorites.append(int(reviews[2].text.replace(',', '')))
                date = footer.find_elements(By.TAG_NAME, "span")[0].text[4:]
                published_dates.append(date)

            else:
                favorites.append(int(reviews[1].text.replace(', ', '')))
                review_counts.append(int(reviews[0].text.replace(',', '')))
                date = footer.find_elements(By.CLASS_NAME, "basicList_etc__LSkN_")[1].text[4:]
                published_dates.append(date)

        except IndexError:
            review_counts.append(0)
            favorites.append(int(reviews[0].text.replace(',', '')))
            date = footer.find_elements(By.TAG_NAME, "span")[0].text[4:]
            published_dates.append(date)

In [30]:
print(len(titles), len(prices), len(review_counts), len(published_dates), len(favorites))
review_counts=review_counts[:-1]
len(review_counts)


200 200 201 200 200


200

In [31]:
result = pd.DataFrame({"Product Name": titles, 
                        "Price": prices,
                        "No._of_reviews": review_counts,
                        "published_date": published_dates,
                        "favorites": favorites})

result

,Product Name,Price,No._of_reviews,published_date,favorites
0,[호환] LG 엘지 공기청정기 필터 퓨리케어 360 AS280DWFC 기본형,14900,3587,2019.12.,1094
1,[호환] LG 엘지 공기청정기 필터 퓨리케어360 일반형 AS247DWE AS181DAW,15900,3214,2019.07.,779
2,[호환] LG 엘지 공기청정기 필터 퓨리케어 일반형,15900,914,2022.01.,365
3,[호환] LG 퓨리케어 공기청정기 필터 엘지 360 헤파 AS280DWFC AS18...,9900,2456,2021.04.,352
4,[호환] LG 엘지 공기청정기 필터 퓨리케어 360,33060,756,2020.08.,89
...,...,...,...,...,...
195,[호환] 국내산 LG H13등급 고성능 AW101UCW 청담필터 엘지 LA-Q,34500,22,2021.10.,2
196,엘지 공기청정기 호환 필터 LA-N151DB Q시리즈 공기청정기필터,44240,39,2022.11.,0
197,엘지 공기청정기 호환 필터 LA-P183DW p set 호환필터 필터교체,68720,21,2022.11.,0
198,엘지 공기청정기 호환 필터 LA-N159DWR Q시리즈 공기청정기호환필터,44570,25,2022.11.,0


In [20]:
result.to_excel(f"naver_shopping_{query}.xlsx")